In [2]:
# 导入warnings这个库，可以忽视一些警告
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)

# 导入一些科学计算相关的库
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import gc
from keras_radam import RAdam
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Keras是用来实现神经网络的框架
from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add,Dropout
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K

Using TensorFlow backend.


In [3]:
# 列出input里面的文件
print(os.listdir("../input"))

['champs-scalar-coupling', 'quantum-machine-9-qm9']


In [4]:
DATA_PATH = '../input/champs-scalar-coupling'
SUBMISSIONS_PATH = './'
# 使用原子的序号来对表示这些原子
ATOMIC_NUMBERS = {
    'H': 1,
    'C': 6,
    'N': 7,
    'O': 8,
    'F': 9
}

In [5]:
# 训练集的数据类型设置
train_dtypes = {
    'molecule_name': 'category',
    'atom_index_0': 'int8',
    'atom_index_1': 'int8',
    'type': 'category',
    'scalar_coupling_constant': 'float32'
}
# 读取训练集文件
train_csv = pd.read_csv(f'{DATA_PATH}/champs-scalar-coupling/train.csv', index_col='id', dtype=train_dtypes)
# 将molecule_name的格式从dsgdb9nsd_xx改成xx方面处理
train_csv['molecule_index'] = train_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
train_csv = train_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant']]
#打印前10个元素
train_csv.head(10)

,molecule_index,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,1,1,0,1JHC,84.807602
1,1,1,2,2JHH,-11.257000
2,1,1,3,2JHH,-11.254800
3,1,1,4,2JHH,-11.254300
4,1,2,0,1JHC,84.807404
5,1,2,3,2JHH,-11.254100
6,1,2,4,2JHH,-11.254800
7,1,3,0,1JHC,84.809303
8,1,3,4,2JHH,-11.254300


In [6]:
# 读取需要提交的文件
submit = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

In [7]:
# 读取测试文件
test_csv = pd.read_csv(f'{DATA_PATH}/test.csv', index_col='id', dtype=train_dtypes)
test_csv['molecule_index'] = test_csv['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')
test_csv = test_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type']]
test_csv.head(10)

,molecule_index,atom_index_0,atom_index_1,type
id,,,,
4658147,4,2,0,2JHC
4658148,4,2,1,1JHC
4658149,4,2,3,3JHH
4658150,4,3,0,1JHC
4658151,4,3,1,2JHC
4658152,15,3,0,1JHC
4658153,15,3,2,3JHC
4658154,15,3,4,2JHH
4658155,15,3,5,2JHH


In [8]:
# 结构数据类型
structures_dtypes = {
    'molecule_name': 'category',
    'atom_index': 'int8',
    'atom': 'category',
    'x': 'float32',
    'y': 'float32',
    'z': 'float32'
}
structures_csv = pd.read_csv(f'{DATA_PATH}/structures.csv', dtype=structures_dtypes)
structures_csv['molecule_index'] = structures_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
structures_csv = structures_csv[['molecule_index', 'atom_index', 'atom', 'x', 'y', 'z']]
structures_csv['atom'] = structures_csv['atom'].replace(ATOMIC_NUMBERS).astype('int8')
structures_csv.head(10)

,molecule_index,atom_index,atom,x,y,z
0,1,0,6,-0.012698,1.085804,0.008001
1,1,1,1,0.002150,-0.006031,0.001976
2,1,2,1,1.011731,1.463751,0.000277
3,1,3,1,-0.540815,1.447527,-0.876644
4,1,4,1,-0.523814,1.437933,0.906397
5,2,0,7,-0.040426,1.024108,0.062564
6,2,1,1,0.017257,0.012545,-0.027377
7,2,2,1,0.915789,1.358745,-0.028758
8,2,3,1,-0.520278,1.343532,-0.775543
9,3,0,8,-0.034360,0.977540,0.007602


In [9]:
# 根据耦合键的类型，提取特定的数据
def build_type_dataframes(base, structures, coupling_type):
    base = base[base['type'] == coupling_type].drop('type', axis=1).copy()
    base = base.reset_index()
    base['id'] = base['id'].astype('int32')
    structures = structures[structures['molecule_index'].isin(base['molecule_index'])]
    return base, structures

In [10]:
# 从structure根据molecule_index和atom_index来得到得到atom的坐标
def add_coordinates(base, structures, index):
    df = pd.merge(base, structures, how='inner',
                  left_on=['molecule_index', f'atom_index_{index}'],
                  right_on=['molecule_index', 'atom_index']).drop(['atom_index'], axis=1)
    df = df.rename(columns={
        'atom': f'atom_{index}',
        'x': f'x_{index}',
        'y': f'y_{index}',
        'z': f'z_{index}'
    })
    return df

In [11]:
# 添加原子的信息
def add_atoms(base, atoms):
    df = pd.merge(base, atoms, how='inner',
                  on=['molecule_index', 'atom_index_0', 'atom_index_1'])
    return df

In [12]:
# 除了原有的atom_index_0和atom_index_1的那些行不加进去，其它的都加进去
def merge_all_atoms(base, structures):
    df = pd.merge(base, structures, how='left',
                  left_on=['molecule_index'],
                  right_on=['molecule_index'])
    df = df[(df.atom_index_0 != df.atom_index) & (df.atom_index_1 != df.atom_index)]
    return df

In [13]:
# 得到中心点的坐标
def add_center(df):
    df['x_c'] = ((df['x_1'] + df['x_0']) * np.float32(0.5))
    df['y_c'] = ((df['y_1'] + df['y_0']) * np.float32(0.5))
    df['z_c'] = ((df['z_1'] + df['z_0']) * np.float32(0.5))

# 得到到中心点的距离
def add_distance_to_center(df):
    df['d_c'] = ((
        (df['x_c'] - df['x'])**np.float32(2) +
        (df['y_c'] - df['y'])**np.float32(2) + 
        (df['z_c'] - df['z'])**np.float32(2)
    )**np.float32(0.5))

# 计算下标suffix1,和suffix2之间距离
def add_distance_between(df, suffix1, suffix2):
    df[f'd_{suffix1}_{suffix2}'] = ((
        (df[f'x_{suffix1}'] - df[f'x_{suffix2}'])**np.float32(2) +
        (df[f'y_{suffix1}'] - df[f'y_{suffix2}'])**np.float32(2) + 
        (df[f'z_{suffix1}'] - df[f'z_{suffix2}'])**np.float32(2)
    )**np.float32(0.5))

In [14]:
# 计算各个原子间的距离
def add_distances(df):
    n_atoms = 1 + max([int(c.split('_')[1]) for c in df.columns if c.startswith('x_')])
    
    for i in range(1, n_atoms):
        for vi in range(min(4, i)):
            add_distance_between(df, i, vi)

In [15]:
# 增加一个特征，该特征为molecule_index的分子对应原子的个数
def add_n_atoms(base, structures):
    dfs = structures['molecule_index'].value_counts().rename('n_atoms').to_frame()
    return pd.merge(base, dfs, left_on='molecule_index', right_index=True)

In [16]:
# 降低内存使用，感觉每个变量的取值范围将其动态改变类型
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [17]:
# 将Categorical类型的数据变成热编码数据
def dummies(df, list_cols):
    for col in list_cols:
        df_dummies = pd.get_dummies(df[col], drop_first=True, 
                                    prefix=(str(col)))
        df = pd.concat([df, df_dummies], axis=1)
    return df

# 添加QM9特征
def add_qm9_features(df):
    # 读取qm9数据
    data_qm9 = pd.read_pickle('../input/quantum-machine-9-qm9/data.covs.pickle')
    # 抛去一些无用的和重复的特征
    to_drop = ['type', 
               'linear', 
               'atom_index_0', 
               'atom_index_1', 
               'scalar_coupling_constant', 
               'U', 'G', 'H', 
               'mulliken_mean', 'r2', 'U0']
    data_qm9 = data_qm9.drop(columns = to_drop, axis=1)
    # 减少内存
    data_qm9 = reduce_mem_usage(data_qm9,verbose=False)
    # 将molecule_name改成molecule_index
    data_qm9['molecule_index'] = data_qm9.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
    data_qm9=data_qm9.drop(columns=['molecule_name'])
    # 将qm9特征加入到df里面去
    df = pd.merge(df, data_qm9, how='left', on=['molecule_index','id'])
    # 抛去molecule_index,id这个对我们预测没啥帮助，此时df已经是最后的训练集或者测试集，特征工程已经处理结束
    df=df.drop(columns=['molecule_index','id'])
    del data_qm9
    gc.collect()
    return df


In [18]:
# 构建需要跑的数据集
def build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=15):
    # 得到base，structures
    base, structures = build_type_dataframes(some_csv, structures_csv, coupling_type)
    # 添加原子1，原子2的坐标
    base = add_coordinates(base, structures, 0)
    base = add_coordinates(base, structures, 1)
    # 扔掉原子1，2的序号的两列
    
    base = base.drop(['atom_0', 'atom_1'], axis=1)
    #  扔掉id这一列
    atoms = base.drop('id', axis=1).copy()
        # 如果有scalar_coupling_constant这一列，则丢掉，scalar_coupling_constant这列是y的值
    if 'scalar_coupling_constant' in some_csv:
        atoms = atoms.drop(['scalar_coupling_constant'], axis=1)
    
    # 添加中心点
    add_center(atoms)
    # 删掉原子1，原子2的坐标，现在用中心点来替代
    atoms = atoms.drop(['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'], axis=1)
    # 合并所有的原子
    atoms = merge_all_atoms(atoms, structures)
    
    # 对所有的原子添加到中心的距离
    add_distance_to_center(atoms)
    
     # 删除中心点位置
    atoms = atoms.drop(['x_c', 'y_c', 'z_c', 'atom_index'], axis=1)
    # 按照molecule_index,atom_index_0,atom_index_1,d_c来对atoms进行排序
    atoms.sort_values(['molecule_index', 'atom_index_0', 'atom_index_1', 'd_c'], inplace=True)
    
    # 提取原子小于n_atoms的分子
    atom_groups = atoms.groupby(['molecule_index', 'atom_index_0', 'atom_index_1'])
    atoms['num'] = atom_groups.cumcount() + 2
    atoms = atoms.drop(['d_c'], axis=1)
    atoms = atoms[atoms['num'] < n_atoms]
    
    # 对索引设置并通过molecule_index展开
    atoms = atoms.set_index(['molecule_index', 'atom_index_0', 'atom_index_1', 'num']).unstack() 
    atoms.columns = [f'{col[0]}_{col[1]}' for col in atoms.columns]
    atoms = atoms.reset_index()
    
    # 转回int8的类型
    for col in atoms.columns:
        if col.startswith('atom_'):
            atoms[col] = atoms[col].fillna(0).astype('int8')
    
    # 转类型
    atoms['molecule_index'] = atoms['molecule_index'].astype('int32')
    # 添加原子信息
    full = add_atoms(base, atoms)
    # 添加距离
    add_distances(full)
    
    # 根据id来进行重新排序
    full.sort_values('id', inplace=True)
    
    return full

In [19]:
# 生成需要使用的label
def take_n_atoms(df, n_atoms, four_start=4):
    labels = []
    for i in range(2, n_atoms):
        label = f'atom_{i}'
        labels.append(label)

    for i in range(n_atoms):
        num = min(i, 4) if i < four_start else 4
        for j in range(num):
            labels.append(f'd_{i}_{j}')
    if 'scalar_coupling_constant' in df:
        labels.append('scalar_coupling_constant')
    labels=['id','molecule_index']+labels
    return df[labels]

In [20]:
# 创建一个神经网络模型
def create_nn_model(input_shape):
    # 输入层
    inp = Input(shape=(input_shape,))
    # 第一层2048个神经元
    x = Dense(2048, activation="relu")(inp)
    # 批归一化
    x = BatchNormalization()(x)
    # 第二层1024个神经元
    x = Dense(1024, activation="relu")(x)
    # 批归一化
    x = BatchNormalization()(x)
    # 第三层512个神经元
    x = Dense(512, activation="relu")(x)
    # 批归一化
    x = BatchNormalization()(x)
    # 输出层
    out = Dense(1, activation="linear")(x)  
    # 定义得到的模型
    model = Model(inputs=inp, outputs=[out])
    return model

In [21]:
# 打印损失函数的变换曲线
def plot_history(history, label):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss for %s' % label)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    _= plt.legend(['Train','Validation'], loc='upper left')
    plt.show()

In [22]:
# 设置GPU选项
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} ) 
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config) 
K.set_session(sess)

In [23]:
from datetime import datetime

# 分子的类型
mol_types=train_csv["type"].unique()
# 交叉验证的分数
cv_score=[]
cv_score_total=0
#  迭代的次数
epoch_n = 700
# verbose为1，表示运行过程中输出信息
verbose = 1
# batch的大小为2048
batch_size = 2048
    
retrain =True

start_time=datetime.now()
test_prediction=np.zeros(len(test_csv))

# 选择那些需要输入神经网络的特征
input_features = ['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7',
       'atom_8','atom_9', 'atom_10','d_1_0', 'd_2_0', 'd_2_1', 'd_3_0',
       'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3', 'd_5_0',
       'd_5_1', 'd_5_2', 'd_5_3', 'd_6_0', 'd_6_1', 'd_6_2', 'd_6_3',
       'd_7_0', 'd_7_1', 'd_7_2', 'd_7_3', 'd_8_0', 'd_8_1', 'd_8_2',
       'd_8_3', 'd_9_0', 'd_9_1', 'd_9_2', 'd_9_3', 'd_10_0', 'd_10_1', 'd_10_2',
       'd_10_3','rc_A', 'rc_B', 'rc_C', 'mu', 'alpha','homo', 'lumo', 
        'gap', 'zpve', 'Cv', 'freqs_min', 'freqs_max','freqs_mean', 'mulliken_min', 
        'mulliken_max', 'mulliken_atom_0','mulliken_atom_1']

# 对于每个分子类型，都训练一个神经网络模型
for mol_type in mol_types:
    
    # 生成模型的保存路径
    model_name_wrt = ('/kaggle/working/molecule_model_%s.hdf5' % mol_type)
    print('Training %s' % mol_type, 'out of', mol_types, '\n')
    
    # 得到数据
    full = build_couple_dataframe(train_csv, structures_csv, mol_type, n_atoms=11)
    full2 = build_couple_dataframe(test_csv, structures_csv, mol_type, n_atoms=11)
    df_train_ = take_n_atoms(full, 11)
    df_train_=add_qm9_features(df_train_)
    df_test_ = take_n_atoms(full2, 11)
    df_test_ = add_qm9_features(df_test_)
    df_train_  = df_train_.fillna(0)
    df_test_  = df_test_.fillna(0)
    
    # 得进行StandardScaler，标准化处理，对于神经网络这种模型得进行这种处理
    # Standard Scaler from sklearn does seem to work better here than other Scalers
    input_data=StandardScaler().fit_transform(pd.concat([df_train_.loc[:,input_features],df_test_.loc[:,input_features]]))   
    #input_data=StandardScaler().fit_transform(df_train_.loc[:,input_features])
    target_data=df_train_.loc[:,"scalar_coupling_constant"].values

    # 切分训练集和测试集
    train_index, cv_index = train_test_split(np.arange(len(df_train_)),random_state=128, test_size=0.1)
    train_target=target_data[train_index]
    cv_target=target_data[cv_index]
    train_input=input_data[train_index]
    cv_input=input_data[cv_index]
    test_input=input_data[len(df_train_):,:]

    # 构建神经网络
    nn_model=create_nn_model(train_input.shape[1])
    
    # If retrain==False, then we load a previous saved model as a starting point.
    if not retrain:
        nn_model = load_model(model_name_rd)
    
    # 神经网络进行编译
    nn_model.compile(loss='mae', optimizer=Adam())#, metrics=[auc])
    
    # EarlyStopping进行回调
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=40,verbose=1, mode='auto', restore_best_weights=True)
    
    # ReduceLROnPlateau作为学习率学习的策略
    rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=30, min_lr=1e-6, mode='auto', verbose=1)
    
    # 根据训练集损失保存最好的模型
    sv_mod = callbacks.ModelCheckpoint(model_name_wrt, monitor='val_loss', save_best_only=True, period=1)
    # 得到历史的学习率记录
    history = nn_model.fit(train_input,[train_target], 
            validation_data=(cv_input,[cv_target]), 
            callbacks=[es, rlr, sv_mod], epochs=epoch_n, batch_size=batch_size, verbose=verbose)
    
    # CV的预测结果
    cv_predict=nn_model.predict(cv_input)
    
    # 画出学习率曲线
    plot_history(history, mol_type)
    accuracy=np.mean(np.abs(cv_target-cv_predict[:,0]))
    
    print(np.log(accuracy))
    cv_score.append(np.log(accuracy))
    cv_score_total+=np.log(accuracy)
    
    # 在训练集上进行预测
    test_predict=nn_model.predict(test_input)
    
    # 把特定类型的分子进行结果赋值
    test_prediction[test_csv["type"]==mol_type]=test_predict[:,0]
    
    # 把session里面的变量清空
    K.clear_session()

cv_score_total/=len(mol_types)


Training 1JHC out of [1JHC, 2JHH, 1JHN, 2JHN, 2JHC, 3JHH, 3JHC, 3JHN]
Categories (8, object): [1JHC, 2JHH, 1JHN, 2JHN, 2JHC, 3JHH, 3JHC, 3JHN] 

Train on 638474 samples, validate on 70942 samples
Epoch 1/700
638474/638474 [==============================] - 6s 10us/step - loss: 57.0505 - val_loss: 22.8552
Epoch 2/700
638474/638474 [==============================] - 4s 7us/step - loss: 1.9682 - val_loss: 2.0453
Epoch 3/700
638474/638474 [==============================] - 4s 7us/step - loss: 1.5750 - val_loss: 1.4695
Epoch 4/700
638474/638474 [==============================] - 4s 7us/step - loss: 1.4251 - val_loss: 1.3254
Epoch 5/700
638474/638474 [==============================] - 4s 7us/step - loss: 1.3411 - val_loss: 1.3037
Epoch 6/700
638474/638474 [==============================] - 4s 7us/step - loss: 1.2919 - val_loss: 1.5403
Epoch 7/700
638474/638474 [==============================] - 4s 7us/step - loss: 1.2497 - val_loss: 1.4760
Epoch 8/700
638474/638474 [=========================

In [24]:
print ('Total training time: ', datetime.now() - start_time)
# 打印出每个分子的结果
i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)

Total training time:  3:15:54.362409
1JHC : cv score is  -0.5566257
2JHH : cv score is  -2.30039
1JHN : cv score is  -1.28141
2JHN : cv score is  -2.0693924
2JHC : cv score is  -1.5288454
3JHH : cv score is  -1.999294
3JHC : cv score is  -1.4811269
3JHN : cv score is  -2.2756944
total cv score is -1.6865973621606827


In [25]:
def submits(predictions):
 
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("/kaggle/working/submission.csv", index=False)
submits(test_prediction)

Add more layers to get a better score! However,maybe,features are really more important than algorithms...